In [24]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [25]:
# Load and preprocess data
def load_data():
    df = pd.read_csv('D:\\ML Folders\\ml_env\\GitHub\\Hero-Product-Recommendation\\Datasets\\rohit_electronics_sales.csv')
    df['Date'] = pd.to_datetime(df['Date'])
    df['Month'] = df['Date'].dt.month
    df['DayOfWeek'] = df['Date'].dt.dayofweek
    df['Revenue_Per_Unit'] = df['Total_Revenue_Incl_GST'] / df['Current Stock'].replace(0, 1)
    df['Popularity'] = df['Rating'] * df['Total_Revenue_Incl_GST']
    df['Stock_Status'] = df['Current Stock'].apply(lambda x: 'Low' if x <= 5 else 'OK')
    return df

In [26]:
# Build and evaluate model
def build_model(df):
    features = ['Product Category', 'Subcategory', 'Brand Name',
                'Unit Price', 'Discount (%)', 'Current Stock',
                'Month', 'DayOfWeek', 'Rating']
    target = 'Revenue_Per_Unit'

    X_train, X_test, y_train, y_test = train_test_split(
        df[features], df[target], test_size=0.2, random_state=42)

    preprocessor = ColumnTransformer([
        ('num', StandardScaler(), ['Unit Price', 'Discount (%)', 'Current Stock', 'Month', 'DayOfWeek', 'Rating']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['Product Category', 'Subcategory', 'Brand Name'])
    ])

    model = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
    ])

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    return model, mae, r2

In [27]:
def get_inputs():
    print("Please enter your filters (press Enter to skip):")
    time_frame = input("Time frame (1-365days): ").strip()
    category = input("Product Category (Mobile, Laptop , Mobile Accessories, Laptop Accessories): ").strip()
    min_price = input("Min Price (₹): ").strip()
    max_price = input("Max Price (₹): ").strip()
    min_rating = input("Minimum Rating (0-5): ").strip()
    include_low_stock = input("Include low stock? (y/n): ").strip().lower()

    return {
        'time_frame': int(time_frame) if time_frame else None,
        'category': category if category else None,
        'min_price': float(min_price) if min_price else 0,
        'max_price': float(max_price) if max_price else float('inf'),
        'min_rating': float(min_rating) if min_rating else 0,
        'include_low_stock': include_low_stock == 'y'
    }

In [28]:
def set_inputs():
    time_frame = 30
    category = 'laptop'
    min_price = 25000
    max_price = 100000
    min_rating = 4.0
    include_low_stock = 'n'

    return {
        'time_frame': int(time_frame) if time_frame else None,
        'category': category if category else None,
        'min_price': float(min_price) if min_price else 0,
        'max_price': float(max_price) if max_price else float('inf'),
        'min_rating': float(min_rating) if min_rating else 0,
        'include_low_stock': include_low_stock == 'y'
    }

In [29]:
def filter_data(df, params):
    filtered = df.copy()

    if params['time_frame']:
        cutoff_date = datetime.now() - timedelta(days=params['time_frame'])
        filtered = filtered[filtered['Date'] >= cutoff_date]

    if params['category']:
        filtered = filtered[filtered['Product Category'].str.contains(params['category'], case=False)]

    filtered = filtered[
        (filtered['Unit Price'] >= params['min_price']) &
        (filtered['Unit Price'] <= params['max_price']) &
        (filtered['Rating'] >= params['min_rating'])
    ]

    if not params['include_low_stock']:
        filtered = filtered[filtered['Stock_Status'] == 'OK']

    return filtered

In [30]:
def calculate_confidence(predictions):
    min_pred, max_pred = predictions.min(), predictions.max()
    return (predictions - min_pred) / (max_pred - min_pred)

In [31]:
def recommend_trending(df, category):
    recent_days = 30
    recent_cutoff = datetime.now() - timedelta(days=recent_days)
    recent_data = df[df['Date'] >= recent_cutoff]

    if category:
        recent_data = recent_data[recent_data['Product Category'].str.contains(category, case=False)]

    trending = recent_data.groupby('Product Name').agg({
        'Total_Revenue_Incl_GST': 'sum',
        'Rating': 'mean',
        'Current Stock': 'mean',
        'Product Category': 'first',
        'Brand Name': 'first',
        'Unit Price': 'mean'
    }).reset_index()

    trending['Trending_Score'] = trending['Total_Revenue_Incl_GST'] * trending['Rating']
    trending_sorted = trending.sort_values(by='Trending_Score', ascending=False).head(5)

    print("\n🔥 Trending Products (last 30 days) in selected category:")
    print(trending_sorted[['Product Name', 'Brand Name', 'Unit Price', 'Rating', 'Total_Revenue_Incl_GST']].to_string(index=False, formatters={
        'Unit Price': '{:,.2f}'.format,
        'Total_Revenue_Incl_GST': '{:,.2f}'.format
    }))

    for _, row in trending_sorted.iterrows():
        avg_stock = row['Current Stock']
        reorder_qty = 20 if avg_stock < 10 else 10
        print(f"🔁 Suggestion: Reorder '{row['Product Name']}' → Qty: {reorder_qty}")

In [32]:
def bcg_matrix(df, category):
    if category:
        df = df[df['Product Category'].str.contains(category, case=False)]

    df['Revenue_Per_Unit'] = df['Total_Revenue_Incl_GST'] / df['Current Stock'].replace(0, 1)
    df['Sales'] = df['Total_Revenue_Incl_GST']
    df['Profit'] = df['Revenue_Per_Unit'] * df['Current Stock']

    stars = df[(df['Sales'] > df['Sales'].median()) & (df['Profit'] > df['Profit'].median())]
    cows = df[(df['Profit'] > df['Profit'].median()) & (df['Sales'] <= df['Sales'].median())]
    question_marks = df[(df['Sales'] <= df['Sales'].median()) & (df['Profit'] > df['Profit'].median())]
    dogs = df[(df['Sales'] <= df['Sales'].median()) & (df['Profit'] <= df['Profit'].median())]

    print("\n🌟 **BCG Matrix Recommendations** 🌟")

    print("\n⭐ **Stars (High Sales, High Profit)**:")
    print(stars[['Product Name', 'Brand Name', 'Unit Price', 'Sales', 'Profit']].head(5).to_string(index=False, formatters={
        'Unit Price': '{:,.2f}'.format,
        'Sales': '{:,.2f}'.format,
        'Profit': '{:,.2f}'.format
    }))

    print("\n🐄 **Cash Cows (High Profit, Stable Sales)**:")
    print(cows[['Product Name', 'Brand Name', 'Unit Price', 'Sales', 'Profit']].head(5).to_string(index=False, formatters={
        'Unit Price': '{:,.2f}'.format,
        'Sales': '{:,.2f}'.format,
        'Profit': '{:,.2f}'.format
    }))

    print("\n❓ **Question Marks (Potential to Grow)**:")
    print(question_marks[['Product Name', 'Brand Name', 'Unit Price', 'Sales', 'Profit']].head(5).to_string(index=False, formatters={
        'Unit Price': '{:,.2f}'.format,
        'Sales': '{:,.2f}'.format,
        'Profit': '{:,.2f}'.format
    }))

    print("\n🐕 **Dogs (Low Sales, Low Profit)**:")
    print(dogs[['Product Name', 'Brand Name', 'Unit Price', 'Sales', 'Profit']].head(5).to_string(index=False, formatters={
        'Unit Price': '{:,.2f}'.format,
        'Sales': '{:,.2f}'.format,
        'Profit': '{:,.2f}'.format
    }))

In [33]:
def main():
    df = load_data()
    model, mae, r2 = build_model(df)

    print(f"\nModel Accuracy Scores:")
    print(f"- Mean Absolute Error: ₹{mae:.2f}")
    print(f"- R² Score: {r2:.2f} (1.0 is perfect)")

    # params = get_inputs()
    params = set_inputs()
    filtered_df = filter_data(df, params)

    if len(filtered_df) == 0:
        print("\nNo products match your criteria!")
        return

    features = ['Product Category', 'Subcategory', 'Brand Name',
                'Unit Price', 'Discount (%)', 'Current Stock',
                'Month', 'DayOfWeek', 'Rating']

    predictions = model.predict(filtered_df[features])
    filtered_df['Predicted_Score'] = predictions
    filtered_df['Confidence'] = calculate_confidence(predictions)

    top_products_grouped = (
        filtered_df.sort_values('Predicted_Score', ascending=False)
        .groupby(['Product Name', 'Brand Name'], as_index=False)
        .first()
    )

    top_5_unique_products = top_products_grouped.sort_values('Predicted_Score', ascending=False).head(5)

    print("\n🔍 Top Hero Product Recommendations:")
    print(top_5_unique_products[['Product Name', 'Brand Name', 'Unit Price', 'Rating', 'Predicted_Score', 'Confidence']].to_string(index=False, formatters={
        'Unit Price': '{:,.2f}'.format,
        'Predicted_Score': '{:,.2f}'.format,
        'Confidence': '{:.2f}'.format
    }))


    see_trending = input("\nWould you like to see trending products and smart reorder suggestions? (y/n): ").strip().lower()
    if see_trending == 'y':
        recommend_trending(df, params['category'])



    see_bcg = input("\nWould you like to see BCG Matrix Product Recommendations? (y/n): ").strip().lower()
    if see_bcg == 'y':
        bcg_matrix(df, params['category'])


    print("\nThank you for using the product recommendation system. Have a great day!")

In [34]:
if __name__ == "__main__":
    main()


Model Accuracy Scores:
- Mean Absolute Error: ₹133.07
- R² Score: 0.98 (1.0 is perfect)

🔍 Top Hero Product Recommendations:
           Product Name Brand Name Unit Price  Rating Predicted_Score Confidence
    Lenovo Ultrabook 16     Lenovo  83,260.00     4.3        3,815.33       1.00
   ASUS Gaming Laptop 6       ASUS  68,567.00     4.7        3,559.95       0.90
        Acer Notebook 5       Acer  60,433.00     4.6        2,288.82       0.41
Lenovo Gaming Laptop 15     Lenovo  51,366.00     4.0        2,193.59       0.37
        HP Ultrabook 38         HP  65,082.00     5.0        1,998.52       0.29

🔥 Trending Products (last 30 days) in selected category:
        Product Name Brand Name Unit Price   Rating Total_Revenue_Incl_GST
     MSI Notebook 43        MSI 178,182.00 3.307692           2,241,315.73
   Dell Ultrabook 39       Dell 158,973.00 2.953846           2,147,884.21
ASUS Gaming Laptop 3       ASUS 175,607.00 3.111111           1,605,926.02
   ASUS Ultrabook 31       ASU